<a href="https://colab.research.google.com/github/NahuelRepetto/Programacion-Concurrente/blob/main/TrabajoN1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EJERCICIO EN C**

In [ ]:
%%writefile fork.c
#include <stdlib.h>
#include <stdio.h>
#include <unistd.h>
#include <sys/wait.h>

int main(int argc, char *argv[]) 
{
  char identidadDelProceso = 'A';  
  int cantHijosA = 3;
  int cantHijosB = 2;  
  int cantHijosC = 0; 
  int cantHijosD = 1; 
  int i,j;
  pid_t pid;  

  for (i=0; i<cantHijosA; i++)      //CREO TODOS LOS HIJOS DEL PROCESO PRINCIPAL
  {    
    pid = fork();    

    if ( pid < 0 )                  //SI HUBO ERROR DE CREACION TERMINO
    {
      printf("Error al crear el nuevo proceso !!!\n");
	    return EXIT_FAILURE;
    }			  

    if( !pid )                    //SI ESTOY EN EL HIJO
    {
      identidadDelProceso = identidadDelProceso + i + 1;  
      printf("Soy el proceso %c con PID: %d. Mi padre es el proceso con PID: %d.\n",identidadDelProceso, getpid(), getppid());	    

      int cantNietos;
      
      if( identidadDelProceso == 'B')
      { 
        cantNietos = cantHijosB;
        identidadDelProceso = 'A' + cantHijosA + 1;
      }
      else if( identidadDelProceso == 'C')
      { 
        cantNietos = cantHijosC;
        identidadDelProceso = 'A' + cantHijosA + cantHijosB+ 1;
      }
      else if( identidadDelProceso == 'D')
      { 
        cantNietos = cantHijosD;
        identidadDelProceso = 'A' + cantHijosA + cantHijosB + cantHijosC + 1;
      }

      for (j=0; j<cantNietos; j++)  //CREO LA CANTIDAD DE NIETOS CORRESPONDIENTE A CADA HIJO
      {                    
        pid = fork(); 

        if ( pid < 0 )
        {
          printf("Error al crear el nuevo proceso !!!\n");
          return EXIT_FAILURE;
        }		

        if( !pid )
        {
          identidadDelProceso = identidadDelProceso + j;
          printf("Soy el proceso %c con PID: %d. Mi padre es el proceso con PID: %d.\n",identidadDelProceso, getpid(), getppid());
          sleep(1000);
          return EXIT_SUCCESS;
        }
      }       
      
      sleep(1000);
  	  return EXIT_SUCCESS;
    }
  }
  
  //SI LLEGO A ESTE PUNTO SOY EL PROCESO PADRE, LOS HIJOS FINALIZAN ANTES 

  printf("Soy el proceso %c con PID: %d. Mi padre es el proceso con PID: %d.\n", identidadDelProceso, getpid(), getppid());
  
  for (i=0; i<cantHijosA; i++)
  {
    wait(NULL);
  } 
    	
  return EXIT_SUCCESS;
}

Writing fork.c


In [ ]:
!gcc -o fork.bin fork.c

In [ ]:
!./fork.bin

In [ ]:
!nohup ./fork.bin 1>salida 2>/dev/null &

In [ ]:
!ps -ef | grep fork.bin

In [ ]:
!kill 493               

In [ ]:
!pstree -pc 493               

fork.bin(493)─┬─fork.bin(494)─┬─fork.bin(497)
              │               └─fork.bin(499)
              ├─fork.bin(495)
              └─fork.bin(496)───fork.bin(498)


# **EJERCICIO EN JAVA**

In [ ]:
%%writefile CreacionDeProcesos.java
import java.io.IOException;

public class CreacionDeProcesos
{
  public static void mostrarInfo(char identidadDelProceso) throws InterruptedException
  {
    ProcessHandle yo = ProcessHandle.current();
    long pid = yo.pid();
    long ppid = yo.parent().get().pid();
    System.out.println("Soy el proceso " + identidadDelProceso + " con PID: " + pid + ". Mi padre es el proceso con PID: " + ppid);
  }   

  public static void main(String[] args) throws IOException, InterruptedException
  {    
    char identidadDelProceso = 'A';
    int cantHijos = 3;
    ProcessBuilder pb;    
    int error;    

    Process [] procesos = new Process[cantHijos];  

    try
    {
      for(int i=0; i<cantHijos; i++)
      {      
        pb = new ProcessBuilder("java", "Hijo.java", Character.toString(identidadDelProceso+1+i)); 
        pb.redirectErrorStream(true);
        pb.inheritIO();  
        procesos [i] = pb.start();
      }     

      mostrarInfo(identidadDelProceso);

      for(int i=0; i<cantHijos; i++)
      {
        error = procesos[i].waitFor(); 
        if( error != 0)
        {
          System.out.println("Error al ejecuar el programa");
          System.exit(0);
        }      
      }
    } 
    catch (IOException e)
    {
      System.err.println("Problema con el start");
    }   
    catch (InterruptedException e)
    {
      System.err.println("Problema con el waitfor");
    }      
  }
}

Overwriting CreacionDeProcesos.java


In [ ]:
%%writefile Hijo.java
import java.io.IOException;
public class Hijo
{
  public static void mostrarInfo(char identidadDelProceso) throws InterruptedException  
  {
    ProcessHandle yo = ProcessHandle.current();
    long pid = yo.pid();
    long ppid = yo.parent().get().pid();
    System.out.println("Soy el proceso " + identidadDelProceso + " con PID: " + pid + ". Mi padre es el proceso con PID: " + ppid);    
    Thread.sleep(100000);
  }

  public static void main(String[] args) throws IOException, InterruptedException 
  {        
    char identidadDelProceso = args[0].toCharArray()[0];
    char nuevaIdentidad = 'A';
    int cantHijosB = 2;  
    int cantHijosC = 0; 
    int cantHijosD = 1;
    int cantHermanos = 3;
    int cantHijos = 0;        
    
    if( identidadDelProceso == 'B' )
    { 
      cantHijos = cantHijosB;     
      nuevaIdentidad = (char) (nuevaIdentidad + cantHermanos);     
    }   
    else if( identidadDelProceso == 'D' )
    { 
      cantHijos = cantHijosD;    
      nuevaIdentidad = (char) (nuevaIdentidad + cantHermanos + cantHijosB + cantHijosC);    
    }   
    else
    {
      cantHijos = 0;   
    }  

    ProcessBuilder pb;    
    int error;    

    Process [] procesos = new Process[cantHijos];  

    try
    {
      for(int i=0; i<cantHijos; i++)
      {      
        pb = new ProcessBuilder("java", "Hijo.java", Character.toString(nuevaIdentidad+1+i)); 
        pb.redirectErrorStream(true);
        pb.inheritIO();  
        procesos [i] = pb.start();
      }     

      mostrarInfo(identidadDelProceso);

      for(int i=0; i<cantHijos; i++)
      {
        error = procesos[i].waitFor(); 
        if( error != 0)
        {
          System.out.println("Error al ejecuar el programa");
          System.exit(0);
        }      
      }
    } 
    catch (IOException e)
    {
      System.err.println("Problema con el start");
    }   
    catch (InterruptedException e)
    {
      System.err.println("Problema con el waitfor");
    }
  }
}

Overwriting Hijo.java


In [ ]:
!chmod 700 ./CreacionDeProcesos.java

In [ ]:
!java ./CreacionDeProcesos.java

In [36]:
!nohup java ./CreacionDeProcesos.java 1>salida 2>/dev/null &

In [ ]:
!ps -ef | grep CreacionDeProcesos.java

In [ ]:
!kill 152459       

In [38]:
!pstree -pT  33797                                            

java(33797)─┬─java(33820)─┬─java(33901)
            │             └─java(33904)
            ├─java(33823)
            └─java(33827)───java(33927)


# **EJERCICIO EN PYTHON**

In [ ]:
%%writefile fork.py
import os
import time

# Crea 3 procesos hijos
for i in range(3):
    pid = os.fork()
    if pid == 0:
        # Este es el proceso hijo
        letra = chr(ord('B')+i)
        print(f'Soy el proceso hijo {letra} con PID {os.getpid()} y mi padre es el proceso con PID {os.getppid()}')
        if i == 0:
            # Este proceso hijo crea dos hijos más
            for j in range(2):
                pid2 = os.fork()
                if pid2 == 0:
                    # Este es el proceso hijo de segundo nivel
                    letra2 = chr(ord('E')+j)
                    print(f'Soy el proceso hijo de segundo nivel {letra2} con PID {os.getpid()} y mi padre es el proceso con PID {os.getppid()}')
                    os._exit(0)
        elif i == 2:
          # Este proceso hijo crea un hijo más
            pid3 = os.fork()
            if pid3 == 0:
              # Este es el proceso hijo de segundo nivel
              print(f"Soy el proceso hijo de segundo nivel G con PID {os.getpid()} y mi padre es el proceso con PID {os.getppid()}")
              os._exit(0)
        time.sleep(30)
        os._exit(0)

# Espera a que todos los procesos hijos terminen
print(f"Soy el proceso padre con PID {os.getpid()} y mi padre es el proceso con PID {os.getppid()}")
for i in range(3):
    print( os.wait())

Overwriting fork.py


In [ ]:
!chmod 700 ./fork.py

In [ ]:
!python ./fork.py

In [39]:
!nohup python ./fork.py 1>salida 2>/dev/null &

In [ ]:
!ps -ef | grep fork.py

In [ ]:
!kill 1937       

In [41]:
!pstree -pc 34009              

python3(34009)─┬─python3(34011)─┬─python3(34014)
               │                └─python3(34015)
               ├─python3(34012)
               └─python3(34013)───python3(34016)
